In [1]:
import torch
import numpy as np
from train import create_dataloader, create_mask
from models import make_transformer_model, make_autoencoder_model
from data_loader import CustomDataset
from utils import get_config_from_json

In [2]:
config = get_config_from_json("./configs/scada1_centralized_with_autoencoder.json")

In [3]:
config['model_path'] = "../experiments/{}/{}/checkpoints/".format(config['experiment'], config['data_dir'])

In [4]:
dataset = CustomDataset(config)
dataloader = create_dataloader(dataset, config)

In [5]:
autoencoder_model = make_autoencoder_model(
    seq_len=config['autoencoder_dims'], d_model=config['d_model'])
autoencoder_model.load_state_dict(torch.load(config['model_path'] + 'best_autoencoder_4.pth'))
autoencoder_model.float()
autoencoder_model.eval()


mask = create_mask(config)
trans_model = make_transformer_model(
    N=6, d_model=dataset.rolling_windows.shape[-1], l_win=config['l_win'], d_ff=128, h=1, dropout=0.1)
trans_model.load_state_dict(torch.load(config['model_path'] + 'best_trans_13.pth'))
trans_model.float()
trans_model.eval()


TransformerModel(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadAttention(
          (linears): ModuleList(
            (0): Linear(in_features=16, out_features=16, bias=True)
            (1): Linear(in_features=16, out_features=16, bias=True)
            (2): Linear(in_features=16, out_features=16, bias=True)
            (3): Linear(in_features=16, out_features=16, bias=True)
          )
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=16, out_features=128, bias=True)
          (w_2): Linear(in_features=128, out_features=16, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm((

In [6]:
%%time
loss = torch.nn.MSELoss()
n_test = dataset.rolling_windows.shape[0]
recon_loss = np.zeros(n_test)
encoder = autoencoder_model.encoder
for i, batch in enumerate(dataloader):
    src = encoder(batch['input'].float())
    trg = encoder(batch['target'].float())
    out = trans_model(src, src_mask=mask)
    for j in range(config['batch_size']):
        try:
            recon_loss[i * config['batch_size'] + j] = loss(
                out[j, config['pre_mask']:config['post_mask'], :], trg[j, config['pre_mask']:config['post_mask'], :])
        except:
            pass


CPU times: user 31min 50s, sys: 8.05 s, total: 31min 58s
Wall time: 3min 18s


In [7]:
recon_loss_old = recon_loss

In [18]:
%%time
loss = torch.nn.MSELoss()
n_test = dataset.rolling_windows.shape[0]
recon_loss = np.zeros(n_test)
encoder = autoencoder_model.encoder
for i, batch in enumerate(dataloader):
    src = encoder(batch['input'].float())
    trg = encoder(batch['target'].float())
    out = trans_model(src, src_mask=mask)
    recon = loss(out[:, config['pre_mask']:config['post_mask'], :], trg[:, config['pre_mask']:config['post_mask'], :])
    
#     recon_loss = np.append(recon_loss, np.array([recon]), axis=0)
#     recon_loss = np.delete(recon_loss, np.s_[:dataset.rolling_windows.shape[0]], axis=0)


CPU times: user 19min 10s, sys: 8.27 s, total: 19min 18s
Wall time: 1min 56s


In [12]:
recon_loss

array([0., 0., 0., ..., 0., 0., 0.])